# Library Download and Load

In [1]:
!pip install transformers
!pip install sentencepiece
!pip install bertviz
!pip install plotly
!pip install pyyaml==5.4.1

     |████████████████████████████████| 4.0 MB 14.6 MB/s 
     |████████████████████████████████| 596 kB 56.7 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 
     |████████████████████████████████| 895 kB 76.4 MB/s 
     |████████████████████████████████| 6.6 MB 76.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 14.3 MB/s 
     |████████████████████████████████| 157 kB 14.3 MB/s 
     |████████████████████████████████| 132 kB 18.6 MB/s 
     |████████████████████████████████| 8.7 MB 74.8 MB/s 
     |████████████████████████████████| 79 kB 10.6 MB/s 
     |████████████████████████████████| 138 kB 79.7 MB/s 
     |████████████████████████████████| 127 kB 75.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3

In [2]:
!pip install wandb

     |████████████████████████████████| 1.8 MB 14.6 MB/s 
     |████████████████████████████████| 144 kB 61.2 MB/s 
     |████████████████████████████████| 181 kB 75.4 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=638a1df551940178e97bd2ae059f80c3792c5896740071b3f5d333f3b2e29d9d
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [4]:
!wandb login

wandb: Currently logged in as: kompactss (use `wandb login --relogin` to force relogin)


In [5]:
import json
import pandas as pd
import numpy as np
import torch
torch.cuda.empty_cache()
from pathlib import Path # 파일시스템의 경로를 단순한 문자열이 아닌 객체로 다루게 하는 것
from torch.utils.data import Dataset, DataLoader, SequentialSampler

from sklearn.model_selection import train_test_split
from termcolor import colored
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap
from torch import nn, optim



from transformers import (
    AdamW,
    AutoModel,
    AutoTokenizer,
    AutoModelForQuestionAnswering
)
from tqdm.auto import tqdm
from bertviz import head_view

from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import gc


import warnings

warnings.filterwarnings(action='ignore') 
from plotly import graph_objs as go

from collections import Counter
import plotly.express as px

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [7]:
import os

DIR = "/content/gdrive/MyDrive/k-digital-goorm-3-korean-mrc/"
TRAIN_SOURCE = os.path.join(DIR, "train.json")
TEST_SOURCE = os.path.join(DIR, "test.json")

with open(TRAIN_SOURCE) as f:
    TRAIN_DATA = json.loads(f.read())
    
with open(TEST_SOURCE) as f:
    TEST_DATA = json.loads(f.read())

In [8]:
train = pd.DataFrame(columns=['guid' ,'title', 'context', 'question' ,'answer_start', 'text'])
guid = 0
for data in TRAIN_DATA['data']:
    for paragraphs in data['paragraphs']:
        for line in paragraphs['qas']:
            train.loc[guid, 'context'] = paragraphs['context']
            train.loc[guid, 'question'] = line['question']
            train.loc[guid, 'answer_start'] = line['answers'][0]['answer_start']
            train.loc[guid, 'text'] = line['answers'][0]['text']            
            train.loc[guid, 'title'] = data['title']
            train.loc[guid, 'guid'] = line['guid']
            guid += 1

In [9]:
test = pd.DataFrame(columns=['guid','title', 'context', 'question' ])
guid = 0
for data in TEST_DATA['data']:
    for paragraphs in data['paragraphs']:
        for line in paragraphs['qas']:
            test.loc[guid, 'context'] = paragraphs['context']
            test.loc[guid, 'question'] = line['question']
            test.loc[guid, 'title'] = data['title']
            test.loc[guid, 'guid'] = line['guid']
            guid += 1

In [ ]:
with open(DIR + '도서.json') as f:
    train_ai_hub_data = json.loads(f.read())

In [ ]:
train_ai_hub_2 = pd.DataFrame(columns=['guid','title', 'context', 'question', 'answer_start', 'text'])
guid = 0
for data in train_ai_hub_data['data']:
    for paragraphs in data['paragraphs']:
        cnt = 0  # 같은 질문은 1개까지만 받음
        for line in paragraphs['qas']:
            if cnt == 1:
                break
            train_ai_hub_2.loc[guid, 'context'] = paragraphs['context']
            train_ai_hub_2.loc[guid, 'question'] = line['question']
            train_ai_hub_2.loc[guid, 'answer_start'] = line['answers'][0]['answer_start']
            train_ai_hub_2.loc[guid, 'text'] = line['answers'][0]['text']
            train_ai_hub_2.loc[guid, 'title'] = data['title']
            train_ai_hub_2.loc[guid, 'guid'] = line['id']
            guid += 1
            cnt += 1
            if guid % 1000 == 0 and guid != 0:
                print('{}개 데이터 저장 완료'.format(guid))

In [ ]:
train_ai_hub_2.to_csv(DIR + 'ai_hub(도서).csv', index=False)

In [10]:
train_ai_hub = pd.read_csv(DIR + 'ai_hub(도서).csv')

In [11]:
train_ai_hub[:10]

,guid,title,context,question,answer_start,text
0,1063333,특별법 이행을 위한 항만 대기오염물질 관리제도 정비 시급,"또한, 정책 및 사업의 본격 추진을 위한 재원의 확보 방안을 마련해야 한다. 특별법...",예산을 분배할 때 사업의 우선 순위를 정해서 차등 지원하는 방법을 뭐라고 하지,260,선택과 집중
1,1063321,특별법 이행을 위한 항만 대기오염물질 관리제도 정비 시급,최근 국민적 관심에 힘입어 미세먼지 등 대기오염 문제가 커다란 사회적 현안으로 등장...,미세먼지 생성물질의 배출을 저감하고 종합적으로 관리하기 위한 법을 어디서 제정했니,335,환경부
2,1063325,특별법 이행을 위한 항만 대기오염물질 관리제도 정비 시급,특별법의 제정으로 항만의 대기오염을 방지·저감하기 위한 정책 및 사업 추진의 법적 ...,특별법의 제정으로 어떤 장소의 대기오염을 방지하기 위한 정책의 법적 근거가 마련되었어,10,항만
3,997733,젠더리뷰 2016년 봄호 제40호,가정폭력방지 및 피해자보호 보호 등에 관한 법률 제4조의 2에 의거하여 3년에 한번...,3년에 한 번씩 이뤄지고 있는 것으로 가족 내 폭력피해를 포괄적으로 조사하는 것은 뭐야,49,가정폭력 실태조사
4,997757,젠더리뷰 2016년 봄호 제40호,아동학대범죄처벌법에서는 아동학대가 발생한 경우에 사법절차의 진행과정에서 피해아동의 ...,아동학대가 발생하면 사법경찰관과 누구에게 응급조치와 임시조치의 권한이 부여되니,67,아동보호전문기관의 직원
5,997681,젠더리뷰 2016년 봄호 제40호,두 번째는 경선방식에 대한 체계적인 전략이 필요하다는 것이다. 20대 총선에서 주요...,경선에서 여성후보들은 자금과 조직에 있어 취약할 경우에는 무엇이 더 유리하니,345,국민여론조사
6,997653,젠더리뷰 2016년 봄호 제40호,6.4지방선거 경선 초반에 세월호 참사가 발생하면서 선거운동이 전면 중단되다시피 한...,당내 경선을 준비하는 후보들에게 있어서 가장 중요한 사항은 뭐야,176,경선일정
7,997737,젠더리뷰 2016년 봄호 제40호,자녀학대 행위자가 남성(아버지)이든 여성(어머니)이든간에 부모에 의한 자녀학대는 용...,성별 고정관념과 역할 구조 하에서 자녀에 대한 돌봄 부담이 거의 전적으로 누구에게 부여돼,131,여성
8,997685,젠더리뷰 2016년 봄호 제40호,여론조사기관의 선정에 있어 특정후보와 연관되지 않은 중립적인 기관을 선정해야 한다....,상향식 공천의 민주성이 실현되기 위해서 무슨 방식 대한 설계와 주의가 요구되지,395,경선방식
9,1283353,젠더리뷰 2016년 봄호 제40호,20대 총선은 코앞으로 다가왔는데 여전히 정당들은 공천문제로 시끄럽다. 새누리당과 ...,새누리당과 새정치민주연합은 하나같이 개혁공천의 하나로 무엇을 주장했어,218,상향식 공천


In [12]:
train = pd.concat([train, train_ai_hub]).reset_index().drop('index', axis=1)
train = train.reset_index().drop('index', axis=1)

In [13]:
# train = train[:30000]  # v1 
train = train[:50000] # v2(koelectra-base)_kykim
# train = train[:100000] # v3(koelectra-base)

In [14]:
def get_answers(x):
    start = x[0]
    text = x[1]
    return {
        'answer_start': [start],
        'text': [text]
    }


train['answers'] = train[['answer_start', 'text']].apply(get_answers, axis=1)

In [15]:
train_df, val_df = train_test_split(train, test_size=0.1, random_state=42)

train_df = train_df.reset_index().drop('index', axis=1)
val_df = val_df.reset_index().drop('index', axis=1)

train_df.shape, val_df.shape

((45000, 7), (5000, 7))

In [16]:
max_length = 512

In [17]:
MODEL_NAME = 'kykim/electra-kor-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/620 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

In [18]:
train_df[:10]['answers']

0        {'answer_start': [222], 'text': ['경찰청']}
1        {'answer_start': [505], 'text': ['동절기']}
2     {'answer_start': [162], 'text': ['국민의 권익']}
3      {'answer_start': [0], 'text': ['국가상징 선양']}
4       {'answer_start': [134], 'text': ['트랙패드']}
5         {'answer_start': [378], 'text': ['개인']}
6    {'answer_start': [62], 'text': ['자립형 복지제도']}
7    {'answer_start': [383], 'text': ['하인리히 법칙']}
8      {'answer_start': [899], 'text': ['통화 기능']}
9         {'answer_start': [294], 'text': ['연금']}
Name: answers, dtype: object

In [19]:
def prepare_train_features(example, tokenizer):
    example["question"] = example["question"].lstrip()
    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=max_length,
        stride=50, 
        return_overflowing_tokens=True, 
        return_offsets_mapping=True,  
        padding="max_length",
    )

    sample_mapping = tokenized_example.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_example.pop("offset_mapping")

    features = []
    for i, offsets in enumerate(offset_mapping):
        feature = {}

        input_ids = tokenized_example["input_ids"][i]
        attention_mask = tokenized_example["attention_mask"][i]

        feature['input_ids'] = input_ids
        feature['attention_mask'] = attention_mask
        feature['offset_mapping'] = offsets

        cls_index = input_ids.index(tokenizer.cls_token_id)
        sequence_ids = tokenized_example.sequence_ids(i)

        sample_index = sample_mapping[i]
        answers = example["answers"]

        if len(answers["answer_start"]) == 0:
            feature["start_position"] = cls_index
            feature["end_position"] = cls_index
        else:
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                feature["start_position"] = cls_index
                feature["end_position"] = cls_index
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                feature["start_position"] = token_start_index - 1
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                feature["end_position"] = token_end_index + 1

        features.append(feature)
    return features

In [20]:
train_features, valid_features = [[] for _ in range(2)]
for i, row in tqdm(train_df.iterrows()):
    train_features += prepare_train_features(row, tokenizer)
for i, row in tqdm(val_df.iterrows()):
    valid_features += prepare_train_features(row, tokenizer)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [21]:
class DatasetRetriever(Dataset):
    def __init__(self, features, mode='train'):
        super(DatasetRetriever, self).__init__()
        self.features = features
        self.mode = mode
        
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, item):   
        feature = self.features[item]
        if self.mode == 'train':
            return {
                'input_ids':torch.tensor(feature['input_ids'] ,dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':torch.tensor(feature['offset_mapping'], dtype=torch.long),
                'start_position':torch.tensor(feature['start_position'], dtype=torch.long),
                'end_position':torch.tensor(feature['end_position'], dtype=torch.long)
            }
        else:
            return {
                'input_ids':torch.tensor(feature['input_ids'], dtype=torch.long),
                'attention_mask':torch.tensor(feature['attention_mask'], dtype=torch.long),
                'offset_mapping':feature['offset_mapping'],
                'sequence_ids':feature['sequence_ids'],
                'id':feature['example_id'],
                'context': feature['context'],
                'question': feature['question']
            }

In [22]:
train_dataset = DatasetRetriever(train_features)
valid_dataset = DatasetRetriever(valid_features)

In [36]:
len(train_dataset), len(valid_dataset)

(48558, 5355)

In [40]:
import wandb
wandb.init(project="Gwang-Yeon(MRC)", 
           entity="goorm-team3",
           config = {
        "learning_rate": 2e-5,
        "epochs": 30,
        "batch_size": 256,
        "accumulation": 16,
        "seed": 42
})
config = wandb.config

train loss,█▄▃▂▂▂▂▂▁▂▂▂▁▂▃▃▄▂▃▄▃▃▁▂▄▂▂▃▂▂▃▃▂▃▁▃▃▂▂▃
train score,█▁▂▂▁▂▂▁▂▂▁▂
valid loss,▄▄▅▄▃▄▄▃▅▅▄▄▇▄▄▂▅█▆▆▇▄▆▇▅▃▆▃▁▇▆▅▄▆▄▄▅▅▃▇
valid score,█▁▁▁▁▁▁▁▁▁▁▁
train loss,10.61604
train score,10.57598
valid loss,10.34206
valid score,10.48725


In [39]:
import os
from statistics import mean

import torch.nn.functional as F
from torch.nn.utils import clip_grad_norm_
from transformers import AutoModelForQuestionAnswering
from transformers import default_data_collator

early_stop_cnt = 0
torch.manual_seed(config.seed)
batch_size = config.batch_size
accumulation = config.accumulation
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)
model.cuda()
data_collator = default_data_collator


train_loader = DataLoader(
    train_dataset, 
    batch_size=batch_size//accumulation, 
    shuffle=True,
    collate_fn=data_collator,
    num_workers=2)

valid_loader = DataLoader(
    valid_dataset,
    batch_size=batch_size//accumulation, 
    shuffle=False, 
    collate_fn=data_collator, 
    num_workers=2)

optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

os.makedirs('dump', exist_ok=True)
train_losses = []
dev_losses = []

step = 0

for epoch in range(1, config.epochs+1):
    print("Epoch", epoch)
    # Training
    running_loss = 0.
    losses = []
    model.train()
    progress_bar = tqdm(train_loader, desc='Train')
    for batch in progress_bar:
        del batch['offset_mapping']
        optimizer.zero_grad(set_to_none=True)
        batch = {key: value.cuda() for key, value in batch.items()}
        start = batch.pop('start_position')
        end = batch.pop('end_position')
        
        output = model(**batch)
        start_logits = output.start_logits
        end_logits = output.end_logits
        loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)
        (loss / accumulation).backward()
        running_loss += loss.item()
        del batch, start, end, start_logits, end_logits, loss
        
        step += 1
        if step % accumulation:
            continue

        clip_grad_norm_(model.parameters(), max_norm=1.)
        optimizer.step()
        losses.append(running_loss / accumulation)
        running_loss = 0.
        progress_bar.set_description(f"Train - Loss: {losses[-1]:.3f}")
        wandb.log({"train loss": losses[-1]})
    train_losses.append(mean(losses))
    print(f"train score: {train_losses[-1]:.3f}")
    wandb.log({"train score": train_losses[-1]})

    # Evaluation
    losses = []
    model.eval()
    for batch in tqdm(valid_loader, desc="Evaluation"):
        del batch['offset_mapping']
        batch = {key: value.cuda() for key, value in batch.items()}
        start = batch.pop('start_position')
        end = batch.pop('end_position')
        
        with torch.no_grad():
            output = model(**batch)
            start_logits = output.start_logits
            end_logits = output.end_logits
        loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)

        losses.append(loss.item())
        wandb.log({"valid loss": losses[-1]})
        del batch, start, end, start_logits, end_logits, loss

    dev_losses.append(mean(losses))
    print(f"Evaluation score: {dev_losses[-1]:.3f}")
    wandb.log({"valid score": dev_losses[-1]})

    if epoch > 2:
        if dev_losses[-1] > dev_losses[-2]:
            early_stop_cnt += 1
    if early_stop_cnt > 5:
        break
model.save_pretrained(f'/content/gdrive/MyDrive/k-digital-goorm-3-korean-mrc/dump/model(koelectra_base(aihub)_v2.{epoch}')

Some weights of the model checkpoint at kykim/electra-kor-base were not used when initializing ElectraForQuestionAnswering: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForQuestionAnswering were not initialized from the model checkpoint at kykim/electra-kor-base and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should prob

Epoch 1


Train:   0%|          | 0/3035 [00:00<?, ?it/s]

RuntimeError: ignored

In [ ]:
gc.collect()

## Inference

In [ ]:
def prepare_test_features(example, tokenizer):
    example["question"] = example["question"].lstrip()
    
    tokenized_example = tokenizer(
        example["question"],
        example["context"],
        truncation="only_second",
        max_length=512,
        stride=50,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    features = []
    for i in range(len(tokenized_example["input_ids"])):
        feature = {}
        feature["example_id"] = example['guid']
        feature['context'] = example['context']
        feature['question'] = example['question']
        feature['input_ids'] = tokenized_example['input_ids'][i]
        feature['attention_mask'] = tokenized_example['attention_mask'][i]
        feature['offset_mapping'] = tokenized_example['offset_mapping'][i]
        feature['sequence_ids'] = [0 if i is None else i for i in tokenized_example.sequence_ids(i)]
        features.append(feature)
    return features

In [ ]:
import collections

def postprocess_qa_predictions(examples, features, raw_predictions, n_best_size = 20, max_answer_length = 20):
    all_start_logits, all_end_logits = raw_predictions
    
    example_id_to_index = {k: i for i, k in enumerate(examples["guid"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    predictions = collections.OrderedDict()

    print(f"Post-processing {len(examples)} example predictions split into {len(features)} features.")

    for example_index, example in examples.iterrows():
        feature_indices = features_per_example[example_index]

        min_null_score = None
        valid_answers = []
        
        context = example["context"]
        for feature_index in feature_indices:
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            sequence_ids = features[feature_index]["sequence_ids"]
            context_index = 1

            features[feature_index]["offset_mapping"] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(features[feature_index]["offset_mapping"])
            ]
            offset_mapping = features[feature_index]["offset_mapping"]
            cls_index = features[feature_index]["input_ids"].index(tokenizer.cls_token_id)
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char: end_char]
                        }
                    )
        
        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[0]
        else:
            best_answer = {"text": "", "score": 0.0}
        
        predictions[example["guid"]] = best_answer["text"]
        
        
    return predictions

In [ ]:
test_features = []
for i, row in test.iterrows():
    test_features += prepare_test_features(row, tokenizer)

In [ ]:
test_dataset = DatasetRetriever(test_features, mode='test')
test_dataloader = DataLoader(
    test_dataset,
    batch_size=16, 
    shuffle=False,
    num_workers=2,
    pin_memory=True, 
    drop_last=False
)

In [ ]:
def get_predictions(trained_model):
    model = trained_model.cuda()
    
    start_logits = []
    end_logits = []
    model.eval()
    for batch in tqdm(test_dataloader):
        with torch.no_grad():
            output = model(batch['input_ids'].cuda(), batch['attention_mask'].cuda())
            start_logits.append(output.start_logits.cpu().numpy().tolist())
            end_logits.append(output.end_logits.cpu().numpy().tolist())
            del output
    gc.collect()
    return np.vstack(start_logits), np.vstack(end_logits)

In [ ]:
start_logits, end_logits = get_predictions(trained_model=model)
# Now Calling Function 

fin_preds = postprocess_qa_predictions(test, test_features, (start_logits, end_logits))

In [ ]:
from string import punctuation
submission = []
for p1, p2 in fin_preds.items():
    p2 = " ".join(p2.split())
    p2 = p2.strip(punctuation)
    submission.append((p1, p2))
    
sample = pd.DataFrame(submission, columns=["id", "Predicted"])

#test_data =pd.merge(left=test_df,right=sample,on='id')

In [ ]:
sample.to_csv('/content/gdrive/MyDrive/k-digital-goorm-3-korean-mrc/out/koelectra_base_aihub_kykim_v2.csv', index=False)
sample = sample.rename({'id':'guid'}, axis='columns')

test_data =pd.merge(left=test,right=sample,on='guid')

In [ ]:
test_data